In [244]:
import pandas as pd
import numpy as np
# import os


# path="/Users/EstebanArgudo/Documents/SECfiles/Files"
# cwd=os.getcwd
# os.chdir(path)

In [245]:
form = '10-K'
year=2009
keyword='LongTermDebtNoncurrent'
variable_list=['LongTermDebtNoncurrent']

# Looping through the years

if (year>= 2009):
    q1_sub = pd.read_csv(str(year)+'q1sub.tsv',sep='\t',encoding = "ISO-8859-1")
    q1_num = pd.read_csv(str(year)+'q1num.tsv',sep='\t',encoding = "ISO-8859-1")

q2_sub = pd.read_csv(str(year)+'q2sub.tsv',sep='\t',encoding = "ISO-8859-1")
q2_num = pd.read_csv(str(year)+'q2num.tsv',sep='\t',encoding = "ISO-8859-1")
q3_sub = pd.read_csv(str(year)+'q3sub.tsv',sep='\t',encoding = "ISO-8859-1")
q3_num = pd.read_csv(str(year)+'q3num.tsv',sep='\t',encoding = "ISO-8859-1")
q4_sub = pd.read_csv(str(year)+'q4sub.tsv',sep='\t',encoding = "ISO-8859-1")
q4_num = pd.read_csv(str(year)+'q4num.tsv',sep='\t',encoding = "ISO-8859-1")

companies=q2_sub.append(q3_sub).append(q4_sub)
values=q2_num.append(q3_num).append(q4_num)

if (year>= 2009):
    companies=companies.append(q1_sub)
    values=values.append(q1_num)

if (year==2018):
    companies=q1_sub
    values=q1_num

C:\Users\josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [246]:
# form = '10-K'
# year=2016
# keyword='ResearchAndDevelopmentExpense'
# variable_list=['ResearchAndDevelopmentExpense']

# # Looping through the years

# if (year> 2009):
#     q1_sub = pd.read_csv(str(year)+'q1sub.csv',encoding = "ISO-8859-1")
#     q1_num = pd.read_csv(str(year)+'q1num.csv',encoding = "ISO-8859-1")

# q2_sub = pd.read_csv(str(year)+'q2sub.csv', encoding = "ISO-8859-1")
# q2_num = pd.read_csv(str(year)+'q2num.csv', encoding = "ISO-8859-1")
# q3_sub = pd.read_csv(str(year)+'q3sub.csv', encoding = "ISO-8859-1")
# q3_num = pd.read_csv(str(year)+'q3num.csv', encoding = "ISO-8859-1")
# q4_sub = pd.read_csv(str(year)+'q4sub.csv', encoding = "ISO-8859-1")
# q4_num = pd.read_csv(str(year)+'q4num.csv', encoding = "ISO-8859-1")

# companies=q2_sub.append(q3_sub).append(q4_sub)
# values=q2_num.append(q3_num).append(q4_num)

# if (year> 2009):
#     companies=companies.append(q1_sub)
#     values=values.append(q1_num)

# if (year==2018):
#     companies=q1_sub
#     values=q1_num

In [247]:
# Complete num file for for 4 quarters with values sorted
values.sort_values(['adsh', 'tag','ddate', 'coreg','value'], inplace=True)
count_values=len(values.index)
count_values

# Complete sub file for for 4 quarters with values sorted
companies.sort_values(['adsh','period','form'], inplace=True)
count_companies=len(companies.index)
count_companies

# Add the cik field to values by merging on adsh
cik_merge=companies[['adsh','cik','fy','sic','ein','period','form']]
val=values.merge(cik_merge,on='adsh',how='left')
len(val)
ciknull=val[val['cik'].isnull()]
count_ciknull=len(ciknull.index)
count_ciknull

# Looping through keywords
# for keyword in variable_list:

# Filter out tha values with tags equal to the keyword
val=val[val['tag']==keyword].loc[:,['adsh','cik','ddate','coreg','value','qtrs']]
val=val.rename(columns={'value':keyword})
count_val=len(val.index)
count_val

############ GET VALUES FROM NUM FILE MATCHING ADSH FROM SUB FILE ##############
comp_form = companies[companies.form == form]
count_total = len(comp_form.index)
count_total

############### NCIK==1
con=comp_form[comp_form['nciks']==1][['adsh','cik','fy','sic','ein','period']]
count_con_total = len(con)
count_con_total

# Add values in NUM to con: Number of entries in con_adsh increase relative to con as multiple values with the same adsh get stacked
con_adsh = con.merge(val[['adsh','ddate','coreg','qtrs',keyword]],on='adsh',how='left')
len(con_adsh.index)
len(con_adsh.drop_duplicates(subset=['adsh'],keep='last'))   # Check

# Subcases:
#1. No adsh in NUM file (null ddate)
con_missing_adsh = con_adsh[con_adsh['ddate'].isnull()]
count_con_subcase1 = len(con_missing_adsh.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase1
#2. adsh in NUM file
con_nonmissing_adsh = con_adsh[con_adsh['ddate'].isnull()==False]
count_con_subcase2 = len(con_nonmissing_adsh.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2
# Subsubcases
# adshs found in NUM file that contain information about the reporting period
con_nonmissing_adsh_reportingperiod=con_nonmissing_adsh[con_nonmissing_adsh.period==con_nonmissing_adsh.ddate]
# adshs found in NUM file that contain information about periods other than the reporting period
con_nonmissing_adsh_otherperiods=con_nonmissing_adsh[con_nonmissing_adsh.period!=con_nonmissing_adsh.ddate]
# 2.a. adshs found that ONLY have records corresponding to OTHER periods and not for reporting period
con_nonmissing_adsh_otherperiods_only=con_nonmissing_adsh_otherperiods[con_nonmissing_adsh_otherperiods.adsh.isin(con_nonmissing_adsh_reportingperiod.adsh)==False]
count_con_subcase2a = len(con_nonmissing_adsh_otherperiods_only.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2a
# 2.b adshs found that ONLY have records corresponding to the reporting period
con_nonmissing_adsh_reportingperiod_only=con_nonmissing_adsh_reportingperiod[con_nonmissing_adsh_reportingperiod.adsh.isin(con_nonmissing_adsh_otherperiods.adsh)==False]
count_con_subcase2b = len(con_nonmissing_adsh_reportingperiod_only.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2b
# 2.c adshs found that have records for BOTH, the reporting period AND other periods
con_nonmissing_adsh_both_periods=con_nonmissing_adsh_reportingperiod[con_nonmissing_adsh_reportingperiod.adsh.isin(con_nonmissing_adsh_otherperiods.adsh)]
count_con_subcase2c = len(con_nonmissing_adsh_both_periods.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2c

# Naturally Null values
con_nonmissing_adsh_reportingperiod_only_null = con_nonmissing_adsh_reportingperiod_only[con_nonmissing_adsh_reportingperiod_only[keyword].isnull()]
con_nonmissing_adsh_both_periods_null = con_nonmissing_adsh_both_periods[con_nonmissing_adsh_both_periods[keyword].isnull()]
count_con_subcase2b_null= len(con_nonmissing_adsh_reportingperiod_only_null.drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2c_null= len(con_nonmissing_adsh_both_periods_null.drop_duplicates(subset=['adsh'],keep='last'))

# Revise count of null values since there are cases where:
# 1. adsh has some subsidiries reporting null values while other subsidiares have non-null values
# 2. adsh has value reported as 0 AND as null.
count_con_subcase2b_null = count_con_subcase2b_null - len(con_nonmissing_adsh_reportingperiod_only[con_nonmissing_adsh_reportingperiod_only.adsh.isin(con_nonmissing_adsh_reportingperiod_only_null.adsh)].drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2c_null = count_con_subcase2c_null - len(con_nonmissing_adsh_both_periods[con_nonmissing_adsh_both_periods.adsh.isin(con_nonmissing_adsh_both_periods_null.adsh)].drop_duplicates(subset=['adsh'],keep='last'))
count_con_subcase2b_null
count_con_subcase2c_null


############### NCIK>1
rest=comp_form[comp_form['nciks']!=1][['adsh','cik','sic','ein','fy','period']]
count_rest_total = len(rest)
count_rest_total
# Add values in NUM to rest: Number of entries in rest_val increase relative to rest as multiple values with the same adsh get stacked
rest_adsh = rest.merge(val[['adsh','ddate','coreg','qtrs',keyword]],on='adsh',how='left')
len(rest_adsh.index)
len(rest_adsh.drop_duplicates(subset=['adsh'],keep='last')) # Check
# Subcases:
#1. No adsh in NUM file (null ddate)
rest_missing_adsh = rest_adsh[rest_adsh['ddate'].isnull()]
count_rest_subcase1 = len(rest_missing_adsh.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase1
#2. adsh in NUM file
rest_nonmissing_adsh = rest_adsh[rest_adsh['ddate'].isnull()==False]
count_rest_subcase2 = len(rest_nonmissing_adsh.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2
# Subsubcases
# adshs found in NUM file that contain information about the reporting period
rest_nonmissing_adsh_reportingperiod=rest_nonmissing_adsh[rest_nonmissing_adsh.period==rest_nonmissing_adsh.ddate]
# adshs found in NUM file that contain information about periods other than the reporting period
rest_nonmissing_adsh_otherperiods=rest_nonmissing_adsh[rest_nonmissing_adsh.period!=rest_nonmissing_adsh.ddate]
# 2.a. adshs found that ONLY have records corresponding to OTHER periods and not for reporting period
rest_nonmissing_adsh_otherperiods_only=rest_nonmissing_adsh_otherperiods[rest_nonmissing_adsh_otherperiods.adsh.isin(rest_nonmissing_adsh_reportingperiod.adsh)==False]
count_rest_subcase2a = len(rest_nonmissing_adsh_otherperiods_only.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2a
# 2.b adshs found that ONLY have records corresponding to the reporting period
rest_nonmissing_adsh_reportingperiod_only=rest_nonmissing_adsh_reportingperiod[rest_nonmissing_adsh_reportingperiod.adsh.isin(rest_nonmissing_adsh_otherperiods.adsh)==False]
count_rest_subcase2b = len(rest_nonmissing_adsh_reportingperiod_only.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2b
# 2.c adshs found that have records for BOTH, the reporting period AND other periods
rest_nonmissing_adsh_both_periods=rest_nonmissing_adsh_reportingperiod[rest_nonmissing_adsh_reportingperiod.adsh.isin(rest_nonmissing_adsh_otherperiods.adsh)]
count_rest_subcase2c = len(rest_nonmissing_adsh_both_periods.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2c

# Naturally Null values
rest_nonmissing_adsh_reportingperiod_only_null = rest_nonmissing_adsh_reportingperiod_only[rest_nonmissing_adsh_reportingperiod_only[keyword].isnull()]
rest_nonmissing_adsh_both_periods_null = rest_nonmissing_adsh_both_periods[rest_nonmissing_adsh_both_periods[keyword].isnull()]
count_rest_subcase2b_null= len(rest_nonmissing_adsh_reportingperiod_only_null.drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2c_null= len(rest_nonmissing_adsh_both_periods_null.drop_duplicates(subset=['adsh'],keep='last'))

# Revise count of null values since there are cases where:
# 1. adsh has some subsidiries reporting null values while other subsidiares have non-null values
# 2. adsh has value reported as 0 AND as null.
count_rest_subcase2b_null = count_rest_subcase2b_null - len(rest_nonmissing_adsh_reportingperiod_only[rest_nonmissing_adsh_reportingperiod_only.adsh.isin(rest_nonmissing_adsh_reportingperiod_only_null.adsh)].drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2c_null = count_rest_subcase2c_null - len(rest_nonmissing_adsh_both_periods[rest_nonmissing_adsh_both_periods.adsh.isin(rest_nonmissing_adsh_both_periods_null.adsh)].drop_duplicates(subset=['adsh'],keep='last'))
count_rest_subcase2b_null
count_rest_subcase2c_null


############ RESULTS
# Tables containing adsh records for which we DIDN't get a value
nonmatched_adsh = con_missing_adsh.append(rest_missing_adsh)
missinginfo_adsh = con_nonmissing_adsh_otherperiods_only.append(rest_nonmissing_adsh_otherperiods_only)
nullvalues_adsh = con_nonmissing_adsh_reportingperiod_only_null.append(con_nonmissing_adsh_both_periods_null).append(rest_nonmissing_adsh_reportingperiod_only_null).append(rest_nonmissing_adsh_both_periods_null )

unsuccess_adsh = nonmatched_adsh.append(missinginfo_adsh).append(nullvalues_adsh)
len(unsuccess_adsh.drop_duplicates(subset=['adsh'],keep='last'))


15

In [248]:
# Tables containing adsh records were we are able to get a value
# Note 1: some records with field_values=0 might also appear as NAN. Keep both.
# Note 2: some coregs have values while others might not, for the same (cik,period) record.
# Hence tables unsuccess_adsh and success_adsh might have common adshs. This potential double counting is taken care of in the stats: there records (adsh) are counted as part of the non-missing non-null records.
con_nonmissing_adsh_reportingperiod_only = con_nonmissing_adsh_reportingperiod_only[con_nonmissing_adsh_reportingperiod_only[keyword].isnull()==False]
con_nonmissing_adsh_both_periods = con_nonmissing_adsh_both_periods[con_nonmissing_adsh_both_periods[keyword].isnull()==False]
rest_nonmissing_adsh_reportingperiod_only = rest_nonmissing_adsh_reportingperiod_only[rest_nonmissing_adsh_reportingperiod_only[keyword].isnull()==False]
rest_nonmissing_adsh_both_periods = rest_nonmissing_adsh_both_periods[rest_nonmissing_adsh_both_periods[keyword].isnull()==False]

success_adsh = con_nonmissing_adsh_reportingperiod_only.append(con_nonmissing_adsh_both_periods).append(rest_nonmissing_adsh_reportingperiod_only).append(rest_nonmissing_adsh_both_periods)
len(success_adsh.drop_duplicates(subset=['adsh'],keep='last'))

16

In [249]:

########### SUMMARY STATS
count_matched = count_con_subcase2 + count_rest_subcase2
count_missing_info = count_con_subcase2a + count_rest_subcase2a
count_nonmissing_info = count_con_subcase2b + count_con_subcase2c + count_rest_subcase2b + count_rest_subcase2c
count_nonmissing_info_null = count_con_subcase2b_null + count_con_subcase2c_null + count_rest_subcase2b_null + count_rest_subcase2c_null
count_total_nonvalues = count_total-count_matched+count_missing_info+count_nonmissing_info_null
count_total_values = count_matched - count_missing_info - count_nonmissing_info_null

# Print summary stats
print(str(year)+" Baseline: Using (adsh) to match records from SUB to NUM Files")
print("Total adsh records in SUB files: "+str(count_total))
print("Total adsh records matched from SUB to NUM files: "+str(count_matched)+"/"+str(count_total))
print("Number adsh records matched with NO info in NUM file about reporting period: "+str(count_missing_info)+"/"+str(count_matched))
print("Number adsh records in NUM files with Null values for "+str(keyword)+": "+str(count_nonmissing_info_null)+"/"+str(count_nonmissing_info))


2009 Baseline: Using (adsh) to match records from SUB to NUM Files
Total adsh records in SUB files: 31
Total adsh records matched from SUB to NUM files: 16/31
Number adsh records matched with NO info in NUM file about reporting period: 0/16
Number adsh records in NUM files with Null values for Liabilities: 0/16


In [250]:
######### GET VALUES FROM NUM FILE MATCHING (CIK,PERIOD) FROM SUB FILE #########
# Drop unsuccess_adsh duplicate records by adsh (i.e. keep list of unmatched adsh or equivalently (cik,period) and keep only relevant fields:
unsuccess_adsh_unique = unsuccess_adsh.drop_duplicates(subset=['adsh'],keep='last')[['adsh','cik','fy','period']]
len(unsuccess_adsh_unique)
len(unsuccess_adsh_unique.drop_duplicates(subset=['cik','period'],keep='last'))

# Left merge similar adsh case, except that now merge is on cik (i.e. adding values in NUM to unsuccess_adsh_unique):
# Number of entries in unsuccess_adsh_unique increases as multiple values with the same cik get stacked.
unsuccess_adsh_newtry = unsuccess_adsh_unique.merge(val[['adsh','cik','ddate','coreg',keyword]],on='cik',how='left')
len(unsuccess_adsh_newtry)
len(unsuccess_adsh_newtry.drop_duplicates(subset=['adsh_x'],keep='last')) # Check

15

In [251]:
# Proceed just like in adsh case for the accounting:
# Subcases:
#1. No (cik,period) record in NUM file (null ddate)
unsuccess_adsh_newtry_missing = unsuccess_adsh_newtry[unsuccess_adsh_newtry['ddate'].isnull()]
count_unsuccess_adsh_newtry_missing = len(unsuccess_adsh_newtry_missing.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_missing
#2. (cik,period) record in NUM file
unsuccess_adsh_newtry_nonmissing = unsuccess_adsh_newtry[unsuccess_adsh_newtry['ddate'].isnull()==False]
count_unsuccess_adsh_newtry_nonmissing = len(unsuccess_adsh_newtry_nonmissing.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing
# Subsubcases
# (cik,period) records found in NUM file that contain information about the reporting period
unsuccess_adsh_newtry_nonmissing_reportingperiod=unsuccess_adsh_newtry_nonmissing[unsuccess_adsh_newtry_nonmissing.period==unsuccess_adsh_newtry_nonmissing.ddate]
len(unsuccess_adsh_newtry_nonmissing_reportingperiod)
# (cik,period) records found in NUM file that contain information about periods other than the reporting period
unsuccess_adsh_newtry_nonmissing_otherperiods=unsuccess_adsh_newtry_nonmissing[unsuccess_adsh_newtry_nonmissing.period!=unsuccess_adsh_newtry_nonmissing.ddate]
len(unsuccess_adsh_newtry_nonmissing_otherperiods)
# Count RECORDS in each partition using adsh_x (record identifier from unsucess_10k_adshunique) easier to use than (cik,period)
# 2.a. (adsh_x) that ONLY have records corresponding to OTHER periods and not for reporting period
unsuccess_adsh_newtry_nonmissing_otherperiods_only=unsuccess_adsh_newtry_nonmissing_otherperiods[unsuccess_adsh_newtry_nonmissing_otherperiods.adsh_x.isin(unsuccess_adsh_newtry_nonmissing_reportingperiod.adsh_x)==False]
count_unsuccess_adsh_newtry_nonmissing_otherperiods_only = len(unsuccess_adsh_newtry_nonmissing_otherperiods_only.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_otherperiods_only
# 2.b (adsh_x) that ONLY have records corresponding to the reporting period
unsuccess_adsh_newtry_nonmissing_reportingperiod_only=unsuccess_adsh_newtry_nonmissing_reportingperiod[unsuccess_adsh_newtry_nonmissing_reportingperiod.adsh_x.isin(unsuccess_adsh_newtry_nonmissing_otherperiods.adsh_x)==False]
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only = len(unsuccess_adsh_newtry_nonmissing_reportingperiod_only.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only
# 2.c (adsh_x) that have records for BOTH, the reporting period AND other periods
unsuccess_adsh_newtry_nonmissing_both_periods=unsuccess_adsh_newtry_nonmissing_reportingperiod[unsuccess_adsh_newtry_nonmissing_reportingperiod.adsh_x.isin(unsuccess_adsh_newtry_nonmissing_otherperiods.adsh_x)]
count_unsuccess_adsh_newtry_nonmissing_both_periods = len(unsuccess_adsh_newtry_nonmissing_both_periods.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_both_periods

# Naturally Null values
unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null = unsuccess_adsh_newtry_nonmissing_reportingperiod_only[unsuccess_adsh_newtry_nonmissing_reportingperiod_only[keyword].isnull()]
unsuccess_adsh_newtry_nonmissing_both_periods_null = unsuccess_adsh_newtry_nonmissing_both_periods[unsuccess_adsh_newtry_nonmissing_both_periods[keyword].isnull()]
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null = len(unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_both_periods_null = len(unsuccess_adsh_newtry_nonmissing_both_periods_null.drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null
count_unsuccess_adsh_newtry_nonmissing_both_periods_null

0

In [252]:
############ RESULTS
# Tables containing (cik,period) records where we are unable to get a value after the new try
newtry_nonmatched_adsh = unsuccess_adsh_newtry_missing
newtry_missinginfo_adsh = unsuccess_adsh_newtry_nonmissing_otherperiods_only
newtry_nullvalues_adsh = unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null.append(unsuccess_adsh_newtry_nonmissing_both_periods_null)

newtry_unsuccess_adsh = newtry_nonmatched_adsh.append(newtry_missinginfo_adsh).append(newtry_nullvalues_adsh)
len(newtry_unsuccess_adsh.drop_duplicates(subset=['cik','period'],keep='last'))

15

In [253]:
# Tables containing (cik,period) records where we are able to get a value
# Note 1: some records with field_values=0 might also appear as NAN. Keep both.
# Note 2: some coregs have values while others might not, for the same (cik,period) record.
# Hence tables newtry_unsuccess_adsh and newtry_success_adsh might have common (cik,period) records. This potential double counting is taken care of in the stats: there records (adsh) are counted as part of the non-missing non-null records.
unsuccess_adsh_newtry_nonmissing_reportingperiod_only = unsuccess_adsh_newtry_nonmissing_reportingperiod_only[unsuccess_adsh_newtry_nonmissing_reportingperiod_only[keyword].isnull()==False]
unsuccess_adsh_newtry_nonmissing_both_periods = unsuccess_adsh_newtry_nonmissing_both_periods[unsuccess_adsh_newtry_nonmissing_both_periods[keyword].isnull()==False]

newtry_success_adsh = unsuccess_adsh_newtry_nonmissing_reportingperiod_only.append(unsuccess_adsh_newtry_nonmissing_both_periods)
len(newtry_success_adsh.drop_duplicates(subset=['cik','period'],keep='last'))


0

In [254]:
# Revise count of null values since there are cases were a cik,period (i.e adsh record) has some subsidiries reporting null values while other subsidiares have non-null values
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null = count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null - len(unsuccess_adsh_newtry_nonmissing_reportingperiod_only[unsuccess_adsh_newtry_nonmissing_reportingperiod_only.adsh_x.isin(unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null.adsh_x)].drop_duplicates(subset=['cik','period'],keep='last'))
count_unsuccess_adsh_newtry_nonmissing_both_periods_null = count_unsuccess_adsh_newtry_nonmissing_both_periods_null - len(unsuccess_adsh_newtry_nonmissing_both_periods[unsuccess_adsh_newtry_nonmissing_both_periods.adsh_x.isin(unsuccess_adsh_newtry_nonmissing_both_periods_null.adsh_x)].drop_duplicates(subset=['cik','period'],keep='last'))

In [255]:
count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null

0

In [256]:
count_unsuccess_adsh_newtry_nonmissing_both_periods_null

0

In [257]:
unsuccess_adsh_newtry_nonmissing_both_periods[unsuccess_adsh_newtry_nonmissing_both_periods.cik.isin(['92679'])]

,adsh_x,cik,fy,period,adsh_y,ddate,coreg,Liabilities


In [258]:
########### SUMMARY STATS
newtry_count_matched = count_unsuccess_adsh_newtry_nonmissing
newtry_count_missing_info = count_unsuccess_adsh_newtry_nonmissing_otherperiods_only
newtry_count_nonmissing_info = count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only + count_unsuccess_adsh_newtry_nonmissing_both_periods
newtry_count_nonmissing_info_null = count_unsuccess_adsh_newtry_nonmissing_reportingperiod_only_null + count_unsuccess_adsh_newtry_nonmissing_both_periods_null
newtry_count_total_values = newtry_count_matched - newtry_count_missing_info - newtry_count_nonmissing_info_null

In [259]:
# Print summary stats
print(str(year)+" Refinement # 1: Using (cik,period) to match records from SUB to NUM Files")
print("Total number of records with no values after baseline (adsh) matching: "+str(count_total_nonvalues)+"/"+str(count_total))
print("Number of records matched using (cik,period): "+str(newtry_count_matched)+"/"+str(count_total_nonvalues))
print("Number of records matched with NO info in NUM file about reporting period: "+str(newtry_count_missing_info)+"/"+str(newtry_count_matched))
print("Number of records matched with info in NUM file but with NULL values for "+str(keyword)+": "+str(newtry_count_nonmissing_info_null)+"/"+str(newtry_count_nonmissing_info))

# Final Summay stats: adsh+(cik,period) mismatching
print("Total number of "+str(keyword)+" values missing: "+str(count_total_nonvalues-newtry_count_matched + newtry_count_missing_info + newtry_count_nonmissing_info_null)+"/"+str(count_total))
print("Total number of "+str(keyword)+" values recovered: "+str(count_total_values+newtry_count_total_values)+"/"+str(count_total))


2009 Refinement # 1: Using (cik,period) to match records from SUB to NUM Files
Total number of records with no values after baseline (adsh) matching: 15/31
Number of records matched using (cik,period): 1/15
Number of records matched with NO info in NUM file about reporting period: 1/1
Number of records matched with info in NUM file but with NULL values for Liabilities: 0/0
Total number of Liabilities values missing: 15/31
Total number of Liabilities values recovered: 16/31


In [260]:
############################## OUTPUT TABLES ###################################
newtry_success_adsh = newtry_success_adsh.rename(columns={'adsh_x':'adsh_10k','adsh_y':'adsh_other'})\
[['adsh_10k','adsh_other','cik','coreg','period','fy','ddate',keyword]]
success_adsh = success_adsh.rename(columns={'adsh':'adsh_10k'})

# data = success_adsh.append(newtry_success_adsh)
# data = success_adsh.append(newtry_success_adsh).append(newtry_unsuccess_adsh)
data = success_adsh.append(newtry_success_adsh)

In [261]:
len(data)

16

In [262]:
data

,Liabilities,adsh_10k,adsh_other,cik,coreg,ddate,ein,fy,period,qtrs,sic
3,7.129000e+09,0000950123-09-057827,NaN,804328,NaN,20090930,953685934,2009,20090930,0,3663
6,2.531100e+09,0000950123-09-064772,NaN,829224,NaN,20090930,911325671,2009,20090930,0,5810
11,8.099100e+09,0000950123-09-066095,NaN,2969,NaN,20090930,231274455,2009,20090930,0,2810
13,1.215700e+07,0000950123-09-066100,NaN,1222333,NaN,20090930,522369757,2009,20090930,0,1040
16,2.479635e+09,0000950123-09-070205,NaN,6951,NaN,20091031,941655526,2009,20091031,0,3674
18,1.259900e+10,0001047469-09-010234,NaN,833444,NaN,20090930,0,2009,20090930,0,7380
20,9.194000e+09,0001047469-09-010297,NaN,1385157,NaN,20090930,980518048,2009,20090930,0,5065
24,5.106000e+09,0001047469-09-010861,NaN,1090872,NaN,20091031,770518772,2009,20091031,0,3825
26,3.631390e+10,0001104659-09-070344,NaN,315189,NaN,20091031,362382580,2009,20091031,0,3523
30,5.020440e+08,0001193125-09-179839,NaN,1002638,NaN,20090630,980154400,2009,20090630,0,7373


In [263]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 3 to 46
Data columns (total 11 columns):
Liabilities    16 non-null float64
adsh_10k       16 non-null object
adsh_other     0 non-null object
cik            16 non-null object
coreg          0 non-null object
ddate          16 non-null object
ein            16 non-null object
fy             16 non-null object
period         16 non-null object
qtrs           16 non-null object
sic            16 non-null object
dtypes: float64(1), object(10)
memory usage: 1.5+ KB


In [264]:
# values = values.rename(columns={'ddate':'period', 'adsh':'adsh_10k'})

In [265]:
# values = values.copy()[values.copy().tag==keyword]

In [266]:
# del data['qtrs']

In [267]:
# fin_data = data.merge(values.loc[:,['adsh_10k','period','qtrs']],on=['adsh_10k','period'],how='left')

In [268]:
# fin_data.info()

In [269]:
# data.to_csv('esteban_data'+str(year)+keyword+'.csv')

In [270]:
data.to_csv('esteban_data'+'_notes_'+str(year)+keyword+'.csv')